In [ ]:
import open3d
import argparse
import sys
sys.path.append('.')
import os
import time
import h5py
import datetime
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn.parallel
import torch.utils.data
from torch.utils.data import DataLoader
import torch.nn.functional as F
from data_utils.ModelNetDataLoader import ModelNetDataLoader, load_data, class_names
from pathlib import Path
from tqdm import tqdm
from utils import test, save_checkpoint, select_avaliable, mkdir
import log
from model.pointnet2 import PointNet2ClsMsg
from model.pointnet import PointNetCls, feature_transform_reguliarzer
import colorsys
from clf import parse_args, root
from adv.chamfer import chamfer_distance_without_batch, chamfer_distance_with_batch

In [ ]:
args = parse_args([])
args.pretrain = 'experiment/weights/clf-pointnet-0.89730-0076.pth'
test_data, test_label = load_data(root, train = False)
testDataset = ModelNetDataLoader(test_data, test_label)
testDataLoader = torch.utils.data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False)

log.debug('Building Model',args.model_name)
if args.model_name == 'pointnet':
    num_class = 40
    model = PointNetCls(num_class,args.feature_transform).cuda()  
else:
    model = PointNet2ClsMsg().cuda()

if args.pretrain is None:
    log.err('No pretrain model')
    exit()

log.debug('Loading pretrain model...')
checkpoint = torch.load(args.pretrain)
model.load_state_dict(checkpoint)
model.eval()
print('Done')

In [ ]:
num = 20
for eps in np.linspace(0,0.1,num=num):
    succ, total = 0,0
    for pts_ch_last, gt in testDataLoader:
        pts_ch_last = pts_ch_last.cuda()
        gt = gt[:, 0].long().cuda()
        
        points = pts_ch_last.transpose(2, 1)
        points.requires_grad = True

        pred, _ = model(points)
        loss = F.nll_loss(pred, gt)
        model.zero_grad()
        loss.backward()
        points_grad = points.grad.data
        pts_adv = points + eps * points_grad.sign()
        pts_adv_ch_last = pts_adv.transpose(2, 1)

        # print(chamfer_distance_with_batch(pts_adv_ch_last, pts_ch_last,False))
        
        output, _ = model(pts_adv)
        adv_chocie = output.data.max(1)[1]

        result = gt == adv_chocie
        succ += result.sum().cpu().detach().numpy()
        total += len(result)

    log.info(eps='%.5f'%(eps),accuracy='%.5f%%'%(succ/total * 100))